In [222]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

np.random.seed(42)

import os
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

from keras.optimizers import Adam
from keras.losses import  MeanSquaredError
from keras.metrics import  RootMeanSquaredError

#SAVING AND LOADING MODEL

In [223]:
df = pd.read_csv('../NSE_data_all_stocks_2022_jan_to_may (1).csv')
df


,Date,Code,Name,12m Low,12m High,Day Low,Day High,Day Price,Previous,Change,Change%,Volume,Adjusted Price
0,3-Jan-22,EGAD,Eaagads Ltd,10,15,13.5,13.8,13.5,13.5,-,-,"4,000",-
1,3-Jan-22,KUKZ,Kakuzi Plc,355,427,385,385,385,385,-,-,-,-
2,3-Jan-22,KAPC,Kapchorua Tea Kenya Plc,80,101,99.5,99.5,99.5,95.5,4,4.19%,100,-
3,3-Jan-22,LIMT,Limuru Tea Plc,260,360,320,320,320,320,-,-,-,-
4,3-Jan-22,SASN,Sasini Plc,16.75,22.6,18.7,18.7,18.7,18.7,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6727,31-May-22,MSC,Mumias Sugar Company Ltd,0.27,0.27,0.27,0.27,0.27,0.27,-,-,-,-
6728,31-May-22,UNGA,Unga Group Ltd,26.1,36.4,29,29,29,30,-1,3.33%,"2,100",-
6729,31-May-22,SCOM,Safaricom Plc,25.5,45.25,25.95,26.45,26,26.25,-0.25,0.95%,"20,079,900",-
6730,31-May-22,FAHR,Stanlib Fahari Income-REIT,5,7.48,5.5,5.6,5.56,5.58,-0.02,0.36%,"11,700",-


In [224]:
#Custom Data cleaner 
import random
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler   

nsc_scaler = StandardScaler()

class NSEDataPreProcessor(BaseEstimator, TransformerMixin):
    
    def __init__(self,company_code ,to_type_cast, to_scale, to_return):
        self.company_code = company_code        
        self.to_type_cast= to_type_cast 
        self.to_return= to_return 
        self.to_scale=to_scale  #same as  columns_to_type_cast but has the company code.
        
        
    def __type_cast_and_scale_xyz_df(self, xyz_data_frame):
        for column_name in self.to_type_cast:
            xyz_data_frame[column_name] = pd.to_numeric(xyz_data_frame[column_name], errors='coerce')
        xyz_data_frame[self.to_scale] = nsc_scaler.fit_transform(xyz_data_frame[self.to_scale])
        
        
        return xyz_data_frame[self.to_return]
    
    def fit(self, X, y=None):
        return self      

    def transform(self, X):  
        
        """
        X - NSC dataFrame
        Selects the data for one company , then names it xyz_company_data_frame 
        NB: Scaler always return columns that are assignable to the dataframe.        
        """        
        
        xyz_company_data_frame = X[X['Code'] == self.company_code].reset_index(drop=True)
        type_converted_xyz_company_df = self.__type_cast_and_scale_xyz_df(
            xyz_company_data_frame       
        )
        
        return type_converted_xyz_company_df
    
# 'Code','Day Price','Day High', 'Day High', '12m High', '12m Low' , 'Day Price','Day High', 'Day High', '12m High', '12m Low'

random_company_code = random.choice(df.Code)

#TPSE, UMME  giving the best results .
    
__nse_dat_preprocessor = NSEDataPreProcessor(
    company_code= 'TPSE',
    to_type_cast=['Day Price','Day High', 'Day Low', '12m High', '12m Low'],
    to_scale=[ 'Previous','Day High', 'Day Low', '12m High', '12m Low'],
    to_return=['Code','Day Price','Previous','Day High', 'Day Low', '12m High', '12m Low'], 
)
xyz_preprocessed_df = __nse_dat_preprocessor.fit_transform(
    df,    
)
display(xyz_preprocessed_df.head(2)) #Show the company being worked on .

preprocessed_df = xyz_preprocessed_df.drop(columns=['Code'])
preprocessed_df
    

,Code,Day Price,Previous,Day High,Day Low,12m High,12m Low
0,TPSE,15.25,1.970978,1.981008,2.034167,0.0,0.0
1,TPSE,15.25,1.970978,1.981008,2.034167,0.0,0.0


,Day Price,Previous,Day High,Day Low,12m High,12m Low
0,15.25,1.970978,1.981008,2.034167,0.0,0.0
1,15.25,1.970978,1.981008,2.034167,0.0,0.0
2,15.25,1.970978,1.981008,2.034167,0.0,0.0
3,15.25,1.970978,1.981008,2.034167,0.0,0.0
4,15.25,1.970978,1.981008,2.034167,0.0,0.0
...,...,...,...,...,...,...
97,12.40,-0.757119,-0.493231,-0.319863,0.0,0.0
98,11.90,-0.421353,-0.840492,-1.063241,0.0,0.0
99,11.90,-0.841060,-0.883900,-1.063241,0.0,0.0
100,11.95,-0.841060,-0.883900,-0.691552,0.0,0.0


In [225]:
x_features = ['Previous','Day High', 'Day Low', '12m High', '12m Low']
x_single_feature =  'Day Price'
y_classes = 'Day Price'

X = preprocessed_df[x_single_feature]
day_price_df = preprocessed_df[x_single_feature]
day_price_df


0      15.25
1      15.25
2      15.25
3      15.25
4      15.25
       ...  
97     12.40
98     11.90
99     11.90
100    11.95
101    11.50
Name: Day Price, Length: 102, dtype: float64

# Using previous 5 days to  make predictions for the sixth day .

In [226]:
def __generate_x_y_from_nsc_df(__data_frame , time_step=5):
    data_array = __data_frame.to_numpy()
    X = []
    y = []
    
    for i in range(len(data_array)-time_step):   
        row = [a for a in data_array[i:i+time_step]]
        label = data_array[i+ time_step]
        X.append(row)
        y.append(label)
    return np.array(X), np.array(y)

features , targets = __generate_x_y_from_nsc_df(day_price_df , time_step=5)

print("X_Data shape : ",features.shape)
print("Y_Data shape : ",targets.shape)


column_names = ['Day1', 'Day2', 'Day3','Day4', 'Day5']


# Create a DataFrame with column names
df = pd.DataFrame(features, columns=column_names)

df['day_six_dayPrice'] = pd.Series(data= targets)
df


X_Data shape :  (97, 5)
Y_Data shape :  (97,)


,Day1,Day2,Day3,Day4,Day5,day_six_dayPrice
0,15.25,15.25,15.25,15.25,15.25,15.25
1,15.25,15.25,15.25,15.25,15.25,15.00
2,15.25,15.25,15.25,15.25,15.00,15.00
3,15.25,15.25,15.25,15.00,15.00,15.00
4,15.25,15.25,15.00,15.00,15.00,14.55
...,...,...,...,...,...,...
92,12.45,12.45,12.00,12.00,12.00,12.40
93,12.45,12.00,12.00,12.00,12.40,11.90
94,12.00,12.00,12.00,12.40,11.90,11.90
95,12.00,12.00,12.40,11.90,11.90,11.95


# Data Splitting : test and train data generation 

In [227]:
def dynamic_data_split(X, y, ratio_train):
    """
    Splits data and labels into training, validation, and test sets.

    Args:
        X (a 2D numpy.ndarray): Data.
        y (numpy.ndarray): Labels.
        ratio_train (float): Ratio for training data.

    Returns:
        tuple: (X_train, y_train, X_val, y_val, X_test, y_test).
    """
    total_samples = X.shape[0]

    train_samples = int(ratio_train * total_samples)  
    
    X_train, y_train = X[:train_samples], y[:train_samples]
    X_test, y_test = X[train_samples:], y[train_samples:]

    return X_train, y_train, X_test, y_test


train_ratio = 0.8
test_ratio = 0.2

features_train, targets_train, features_test, targets_test = dynamic_data_split(features,targets , train_ratio)

features_train.shape,targets_train.shape, features_test.shape, targets_test.shape

((77, 5), (77,), (20, 5), (20,))

In [228]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.datasets import make_regression
import xgboost

# Cross Validating Linear Regression model and XGBoost Regression.
### -------->>>> Allows us get a quick estimate on which model performs better on our dataset.
## 1.Linear Regressor

In [229]:
model = LinearRegression()

# Perform k-fold cross-validation for the L. regression model (let's say k=5)
cross_val_scores = cross_val_score(model, features, targets, cv=5, scoring='neg_mean_squared_error')

print("Linear Regression Cross-Validation Scores:", end=' ')
print(cross_val_scores)

average_score = np.mean(cross_val_scores)
print(f"\nAverage Performance Metric: {average_score:.2f}")


Linear Regression Cross-Validation Scores: [-0.51828255 -0.17242521 -0.13913003 -0.104693   -0.1590361 ]

Average Performance Metric: -0.22


## 2. Extreme Gradient Boosting Regressor

In [230]:

model = xgboost.XGBRegressor(objective='reg:squarederror')  # Specifies regression objective

cross_val_scores = cross_val_score(model, features, targets, cv=5, scoring='neg_mean_squared_error')

print("Gradient Boosting Regressor Cross-Validation Scores:", end=' ')
print(cross_val_scores)

average_score = np.mean(cross_val_scores)
print(f"\nAverage Performance Metric: {average_score:.2f}")


Gradient Boosting Regressor Cross-Validation Scores: [-1.0550983  -0.35816781 -0.35275916 -0.13476207 -0.1505617 ]

Average Performance Metric: -0.41


In [ ]:
base_model = xgboost.XGBRegressor(objective='reg:squarederror', random_state=42)
hyper_parameter_grid ={
    'n_estimators': [100, 200, 300,500,1000],
    'learning_rate': [0.01,0.15, 0.1, 0.2],
    'booster':['gbtree','gblinear'],
    'max_depth': [3, 5,7,15],
    'min_child_weight': [1, 3,5,7,9,10],

}
grid_search =RandomizedSearchCV(
    estimator=base_model, 
    param_distributions= hyper_parameter_grid,
    cv=5,
    n_iter=100, 
    scoring='neg_mean_squared_error', 
    n_jobs=-1
)
# grid_search =GridSearchCV(estimator=base_model, param_grid= hyper_parameter_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

grid_search.fit(features_train, targets_train)


/home/kering/Desktop/MachineLearningProjects/KnowledgeBasedSystemsML/KnowledgeBasedSystemsVenv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [17:25:38] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "max_depth", "min_child_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/home/kering/Desktop/MachineLearningProjects/KnowledgeBasedSystemsML/KnowledgeBasedSystemsVenv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [17:25:38] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "max_depth", "min_child_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/home/kering/Desktop/MachineLearningProjects/KnowledgeBasedSystemsML/KnowledgeBasedSystemsVenv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [17:25:38] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "max_depth", "min_child_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/home/kering/Desktop/MachineLearningProjects/KnowledgeBasedSystemsML/Knowled

In [ ]:
best_params = grid_search.best_params_
best_params

In [ ]:
grid_search_cv_best_results = {
    'colsample_bytree': 1.0,
    'learning_rate': 0.01,
    'max_depth': 5,
    'min_child_weight': 5,
    'n_estimators': 300,
    'subsample': 1.0
}
final_model = xgboost.XGBRegressor(objective='reg:squarederror', random_state=42, ** grid_search_cv_best_results)
final_model.fit(features_train, targets_train)

In [ ]:
import pandas as pd

predicted_stock_prices = final_model.predict(features_test)
resultDf = pd.DataFrame(data = {
    'Actual':targets_test,
    'Predictions':predicted_stock_prices.flatten(),    
})
resultDf

In [ ]:
import matplotlib.pyplot as plt 
plt.plot(targets_test, color = 'red', label = 'Actual Stock Price')
plt.plot(predicted_stock_prices, color = 'green', label = 'Predicted Stock Price')
plt.title(random_company_code + ' Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.legend()
plt.show()
